In [ ]:
# ─── Test Block: Fixed Risk Calculations ─────────────────────────────────────

# Define exact window
start = "2019-04-30"
end   = "2024-03-31"

# Fetch and compute stock returns
stock_prices  = fetch_monthly_close("PCTY", start_date=start, end_date=end)
stock_returns = calc_monthly_returns(stock_prices)

# Helper: align any factor return series to stock_returns
def stable_align(series: pd.Series) -> pd.Series:
    return series.loc[stock_returns.index.intersection(series.index)]

# Fetch and persist factor return series over the same window
spy_prices      = fetch_monthly_close("SPY", start_date=start, end_date=end)
spy_returns     = calc_monthly_returns(spy_prices)

momentum_raw    = fetch_excess_return(
    "MTUM", "SPY", start_date=start, end_date=end
)
value_raw       = fetch_excess_return(
    "IWD",  "SPY", start_date=start, end_date=end
)
industry_raw    = fetch_peer_median_monthly_returns(
    ["XSW"], start_date=start, end_date=end
)
subindustry_raw = fetch_peer_median_monthly_returns(
    ["PAYC","PYCR","CDAY","ADP","PAYX","WDAY" ], start_date=start, end_date=end
)

# Align all series to stock_returns
market_returns    = stable_align(spy_returns)
momentum_returns  = stable_align(momentum_raw)
value_returns     = stable_align(value_raw)
industry_returns  = stable_align(industry_raw)
subind_returns    = stable_align(subindustry_raw)

# Build DataFrame for base market regression
df_ret = pd.DataFrame({
    "stock":  stock_returns,
    "market": market_returns
}).dropna()

# Run and print base risk stats
print("n_obs:", len(df_ret))
print("Volatility Metrics:", compute_volatility(df_ret["stock"]))
print("Risk Metrics:",       compute_regression_metrics(df_ret))

# Build factor dictionary and run single-factor regressions
factor_dict = {
    "market":      market_returns,
    "momentum":    momentum_returns,
    "value":       value_returns,
    "industry":    industry_returns,
    "subindustry": subind_returns
}

df_factors_summary = compute_factor_metrics(stock_returns, factor_dict)
print("Single-Factor Regression Summary:\n", df_factors_summary)

In [ ]:
# 1) Define window, weights, expected returns, and per-stock factor proxies
start_date = "2019-05-31"
end_date   = "2024-03-31"

#weights    = {"TW":0.15, "MSCI":0.15, "NVDA":0.17, "PCTY":0.15, "AT.L":0.28}
weights=parsed["weights"]

expected_returns = {"TW":0.15,"MSCI":0.16,"NVDA":0.20,"PCTY":0.17,"AT.L":0.25,"SHV":0}

# stock_factor_proxies maps each ticker to the ETFs/peer‐lists you want to use:
stock_factor_proxies = {
    "TW":   {"market":"SPY","momentum":"MTUM","value":"IWD","industry":"KCE","subindustry":["TW","MSCI","NVDA"]},
    "MSCI": {"market":"SPY","momentum":"MTUM","value":"IWD","industry":"KCE","subindustry":["TW","MSCI","NVDA"]},
    "NVDA": {"market":"SPY","momentum":"MTUM","value":"IWD","industry":"SOXX","subindustry":["SOXX","XSW","IXC"]},
    "PCTY": {"market":"SPY","momentum":"MTUM","value":"IWD","industry":"XSW","subindustry":["PAYC","CDAY","ADP"]},
    "AT.L": {"market":"ACWX","momentum":"IMTM","value":"IVLU","industry":"IXC","subindustry":["IXC"]},
    "SHV": {"market":"SPY","momentum":"IMTM","value":"IWD","industry":"AGG","subindustry":["SHY"]}

}

In [ ]:
# File: run_portfolio_risk.py

# Portfolio inputs

# 1) Define window, weights, expected returns, and per-stock factor proxies
start_date = "2019-05-31"
end_date   = "2024-03-31"

portfolio_input = {
    "TW":   {"weight": 0.15},
    "MSCI": {"weight": 0.15},
    "NVDA": {"weight": 0.17},
    "PCTY": {"weight": 0.15},
    "AT.L": {"weight":0.28},
    "SHV": {"weight":0.10} #proxy for cash
}

#portfolio_input = {
#    "TW":   {"shares": 100},
#    "MSCI": {"dollars": 15000},
#    "NVDA": {"shares": 50},
#    "PCTY": {"dollars": 10000},
#    "AT.L": {"shares": 300}
#}

parsed = standardize_portfolio_input(portfolio_input, latest_price)

#weights    = {"TW":0.15, "MSCI":0.15, "NVDA":0.17, "PCTY":0.15, "AT.L":0.28}
weights=parsed["weights"]

print("=== Normalized Weights ===")
print(parsed["weights"])

print("\n=== Dollar Exposure ===")
print(parsed["dollar_exposure"])

print("\n=== Total Portfolio Value ===")
print(parsed["total_value"])

print("\n=== Net Exposure (sum of weights) ===")
print(parsed["net_exposure"])

print("\n=== Gross Exposure (sum of abs(weights)) ===")
print(parsed["gross_exposure"])

print("\n=== Leverage (gross / net) ===")
print(parsed["leverage"])


In [ ]:
# File: run_portfolio_risk.py

# YAML File Loader for Expectations and Factor Proxies
import yaml
from pprint import pprint  # cleaner multiline dict printing

# Load YAML config
with open("portfolio.yaml", "r") as f:
    expectations = yaml.safe_load(f)

expected_returns = expectations["expected_returns"]
stock_factor_proxies = expectations["stock_factor_proxies"]

print("=== Expected Returns ===")
pprint(expected_returns)

print("\n=== Stock Factor Proxies ===")
for ticker, proxies in stock_factor_proxies.items():
    print(f"\n→ {ticker}")
    pprint(proxies)

In [ ]:
# File: run_portfolio_risk.py

import yaml
from pprint import pprint

# Load YAML config
with open("portfolio.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extract inputs
start_date            = config["start_date"]
end_date              = config["end_date"]
portfolio_input       = config["portfolio_input"]
expected_returns      = config["expected_returns"]
stock_factor_proxies  = config["stock_factor_proxies"]

# Standardize inputs
parsed  = standardize_portfolio_input(portfolio_input, latest_price)
weights = parsed["weights"]

# Output
print("=== Normalized Weights ===")
print(weights)

print("\n=== Dollar Exposure ===")
print(parsed["dollar_exposure"])

print("\n=== Total Portfolio Value ===")
print(parsed["total_value"])

print("\n=== Net Exposure (sum of weights) ===")
print(parsed["net_exposure"])

print("\n=== Gross Exposure (sum of abs(weights)) ===")
print(parsed["gross_exposure"])

print("\n=== Leverage (gross / net) ===")
print(parsed["leverage"])

In [ ]:
# File: run_portfolio_risk.py

# ─── Run Portfolio View ─────────────────────────────────────────────────

import pandas as pd

# 2) Call the high-level summary builder
summary = build_portfolio_view(
    weights,
    start_date,
    end_date,
    expected_returns=expected_returns,
    stock_factor_proxies=stock_factor_proxies
)

# 3) Unpack and print what you need
print("=== Target Allocations ===")
print(summary["allocations"], "\n")

print("=== Portfolio Returns (head) ===")
print(summary["portfolio_returns"].head(), "\n")

print("=== Covariance Matrix ===")
print(summary["covariance_matrix"], "\n")

print("=== Correlation Matrix ===")
print(summary["correlation_matrix"], "\n")

print(f"Monthly Volatility:  {summary['volatility_monthly']:.4%}")
print(f"Annual Volatility:   {summary['volatility_annual']:.4%}\n")

print("=== Risk Contributions ===")
print(summary["risk_contributions"], "\n")

print("Herfindahl Index:", summary["herfindahl"], "\n")

print("=== Per-Stock Factor Betas ===")
print(summary["df_stock_betas"], "\n")

print("=== Portfolio-Level Factor Betas ===")
print(summary["portfolio_factor_betas"], "\n")

print("=== Per-Asset Vol & Var ===")
print(summary["asset_vol_summary"], "\n")

# ── NEW: factor-level diagnostics ─────────────────────────────────────────
print("=== Factor Annual Volatilities (σ_i,f) ===")
print(summary["factor_vols"].round(4))          # pretty table in Jupyter

print("\n=== Weighted Factor Variance   w_i² · β_i,f² · σ_i,f² ===")
print(summary["weighted_factor_var"].round(6), "\n")

print("=== Portfolio Variance Decomposition ===")
var_dec = summary["variance_decomposition"]

print(f"Portfolio Variance:          {var_dec['portfolio_variance']:.4f}")
print(f"Idiosyncratic Variance:      {var_dec['idiosyncratic_variance']:.4f}  ({var_dec['idiosyncratic_pct']:.0%})")
print(f"Factor Variance:             {var_dec['factor_variance']:.4f}  ({var_dec['factor_pct']:.0%})\n")

print("=== Factor Variance (absolute) ===")
for k, v in var_dec["factor_breakdown_var"].items():
    print(f"{k.title():<10} : {v:.5f}")

# Optional: exclude 'industry' and 'subindustry' from factor breakdown
filtered = {
    k: v for k, v in var_dec["factor_breakdown_pct"].items()
    if k not in ("industry", "subindustry")
    }

#print("\n=== Factor Variance (% of Portfolio) ===")
#for k, v in var_dec["factor_breakdown_pct"].items():
    #print(f"{k.title():<10} : {v:.0%}")

print("\n=== Factor Variance (% of Portfolio, excluding industry) ===")
for k, v in filtered.items():
    print(f"{k.title():<10} : {v:.0%}")

print("\n=== Industry Variance (absolute) ===")
for k, v in summary["industry_variance"]["absolute"].items():
    print(f"{k:<10} : {v:.6f}")

print("\n=== Industry Variance (% of Portfolio) ===")
for k, v in summary["industry_variance"]["percent_of_portfolio"].items():
    print(f"{k:<10} : {v:.1%}")

In [ ]:
# File: run_portfolio_risk.py

import yaml
from pprint import pprint

# ─── Load Portfolio Config ─────────────────────────────────────────────
with open("portfolio.yaml", "r") as f:
    config = yaml.safe_load(f)

# Extract inputs
start_date            = config["start_date"]
end_date              = config["end_date"]
portfolio_input       = config["portfolio_input"]
expected_returns      = config["expected_returns"]
stock_factor_proxies  = config["stock_factor_proxies"]

# ─── Standardize Portfolio Weights ─────────────────────────────────────
parsed  = standardize_portfolio_input(portfolio_input, latest_price)
weights = parsed["weights"]

# ─── Display Inputs and Exposures ──────────────────────────────────────
print("=== Normalized Weights ===")
print(weights)

print("\n=== Dollar Exposure ===")
print(parsed["dollar_exposure"])

print("\n=== Total Portfolio Value ===")
print(parsed["total_value"])

print("\n=== Net Exposure (sum of weights) ===")
print(parsed["net_exposure"])

print("\n=== Gross Exposure (sum of abs(weights)) ===")
print(parsed["gross_exposure"])

print("\n=== Leverage (gross / net) ===")
print(parsed["leverage"])

# ─── Display Expectations and Factor Proxies ───────────────────────────
print("\n=== Expected Returns ===")
pprint(expected_returns)

print("\n=== Stock Factor Proxies ===")
for ticker, proxies in stock_factor_proxies.items():
    print(f"\n→ {ticker}")
    pprint(proxies)

In [ ]:
def run_portfolio(filepath: str):
    """
    Loads a YAML portfolio config file, standardizes weights,
    builds risk profile, and prints key decomposition.
    """
    with open(filepath, "r") as f:
        config = yaml.safe_load(f)

    weights = standardize_portfolio_input(config["portfolio_input"], latest_price)["weights"]

    summary = build_portfolio_view(
        weights,
        config["start_date"],
        config["end_date"],
        config.get("expected_returns"),
        config.get("stock_factor_proxies")
    )

    display_portfolio_summary(summary)

In [ ]:
# === Evaluate Portfolio Risk Limits ===
from pprint import pprint

# Load limits from YAML
vol_limit   = risk_config["portfolio_limits"]["max_volatility"]
loss_limit  = risk_config["portfolio_limits"]["max_loss"]
weight_limit = risk_config["concentration_limits"]["max_single_stock_weight"]
var_limits = risk_config["variance_limits"]

# Extract data from summary
annual_vol = summary["volatility_annual"]
weights    = summary["allocations"]["Portfolio Weight"]
variance_decomp = summary["variance_decomposition"]

# 1. Volatility
vol_pass = annual_vol <= vol_limit

# 2. Loss Limit – placeholder (depends on expected drawdown modeling)
loss_pass = True  # not implemented yet

# 3. Concentration
max_weight = weights.abs().max()
weight_pass = max_weight <= weight_limit

# 4. Variance Contributions
factor_var_pct = variance_decomp["factor_pct"]
market_var_pct = variance_decomp["factor_breakdown_pct"].get("market", 0.0)

industry_pct_dict = summary["industry_variance"]["percent_of_portfolio"]
industry_pct_max  = max(industry_pct_dict.values()) if industry_pct_dict else 0.0

factor_pass  = factor_var_pct  <= var_limits["max_factor_contribution"]
market_pass  = market_var_pct  <= var_limits["max_market_contribution"]
industry_pass = industry_pct_max <= var_limits["max_industry_contribution"]

# === Summary Output ===
print("=== Portfolio Risk Limit Checks ===")
print(f"Volatility:             {annual_vol:.2%}  ≤ {vol_limit:.2%}     → {'PASS' if vol_pass else 'FAIL'}")
print(f"Max Weight:             {max_weight:.2%}  ≤ {weight_limit:.2%}  → {'PASS' if weight_pass else 'FAIL'}")
print(f"Factor Var %:           {factor_var_pct:.2%}  ≤ {var_limits['max_factor_contribution']:.2%} → {'PASS' if factor_pass else 'FAIL'}")
print(f"Market Var %:           {market_var_pct:.2%}  ≤ {var_limits['max_market_contribution']:.2%} → {'PASS' if market_pass else 'FAIL'}")
print(f"Top Industry Var %:     {industry_pct_max:.2%}  ≤ {var_limits['max_industry_contribution']:.2%} → {'PASS' if industry_pass else 'FAIL'}")

In [ ]:
# File: risk_helpers.py

from typing import Dict, Tuple

def infer_max_betas_from_losses(
    worst_factor_losses: Dict[str, Tuple[str, float]],
    loss_limit_pct: float
) -> Dict[str, float]:
    """
    Infers max allowable beta per factor to stay within portfolio loss limit.

    Args:
        worst_factor_losses (Dict[str, Tuple[str, float]]): 
            {factor_type: (proxy, worst_loss)}, e.g. {'market': ('SPY', -0.13)}
        loss_limit_pct (float): Target portfolio loss limit in decimal (e.g., 0.10 for -10%)

    Returns:
        Dict[str, float]: {factor_type: max_beta}
    """
    max_betas = {}

    for factor, (_, worst_return) in worst_factor_losses.items():
        if worst_return == 0:
            continue
        max_beta = loss_limit_pct / abs(worst_return)
        max_betas[factor] = round(max_beta, 3)

    return max_betas

In [ ]:
# File: run_risk.py

# === TEST: Calculate Max Beta Limits ===

import yaml
import pandas as pd
from datetime import datetime
from pprint import pprint

# ─── 1. Load Portfolio + Risk Config ────────────────────────────────────────
with open("portfolio.yaml", "r") as f:
    portfolio_config = yaml.safe_load(f)

with open("risk_limits.yaml", "r") as f:
    risk_config = yaml.safe_load(f)

stock_factor_proxies = portfolio_config["stock_factor_proxies"]
LOSS_LIMIT = risk_config["max_single_factor_loss"]  # e.g. -0.10

# ─── 2. Set 10-Year Window ──────────────────────────────────────────────────
end_date = datetime.today().strftime("%Y-%m-%d")
start_date = (datetime.today() - pd.DateOffset(years=10)).strftime("%Y-%m-%d")

# ─── 3. Compute Worst Per-Proxy Losses ──────────────────────────────────────
print("=== Worst Monthly Losses per Proxy ===")
worst_losses = get_worst_monthly_factor_losses(stock_factor_proxies, start_date, end_date)
for proxy, loss in sorted(worst_losses.items(), key=lambda x: x[1]):
    print(f"{proxy:<12} : {loss:.2%}")

# ─── 4. Aggregate to Worst per Factor Type ──────────────────────────────────
print("\n=== Worst Monthly Losses per Factor Type ===")
worst_by_factor_type = aggregate_worst_losses_by_factor_type(stock_factor_proxies, worst_losses)
for factor_type, (proxy, loss) in worst_by_factor_type.items():
    print(f"{factor_type:<10} → {proxy:<12} : {loss:.2%}")

# ─── 5. Compute Max Allowable Beta per Factor ───────────────────────────────
print(f"\n=== Max Allowable Beta per Factor Type (Loss Limit = {LOSS_LIMIT:.0%}) ===")
for factor_type, (_, worst_return) in worst_by_factor_type.items():
    if worst_return >= 0:
        max_beta = float('inf')  # No loss → unrestricted
    else:
        max_beta = LOSS_LIMIT / worst_return
    print(f"{factor_type:<10} → β ≤ {max_beta:.2f}")

max_betas_dict = {
    factor: float('inf') if worst_ret >= 0 else LOSS_LIMIT / worst_ret
    for factor, (_, worst_ret) in worst_by_factor_type.items()
}

In [ ]:
def run_portfolio(filepath: str):
    """
    High-level “one-click” entry-point for a full portfolio risk run.

    It ties together **all** of the moving pieces you’ve built so far:

        1.  Loads the portfolio YAML file (positions, dates, factor proxies).
        2.  Loads the firm-wide risk-limits YAML.
        3.  Standardises the raw position inputs into weights, then calls
            `build_portfolio_view` to produce the master `summary` dictionary
            (returns, vol, correlation, factor betas, variance decomposition, …).
        4.  Pretty-prints the standard risk summary via `display_portfolio_summary`.
        5.  Derives *dynamic* max-beta limits:
                • looks back over the analysis window,
                • finds worst 1-month drawdowns for every unique factor proxy,
                • converts those losses into a per-factor β ceiling
                  using the global `max_single_factor_loss`.
        6.  Runs two rule-checkers
                • `evaluate_portfolio_risk_limits`   →   Vol, concentration, factor %
                • `evaluate_portfolio_beta_limits`   →   Actual β vs. max β
        7.  Prints both tables in a compact “PASS/FAIL” console report.

    Parameters
    ----------
    filepath : str
        Path to the *portfolio* YAML ( **not** the risk-limits file ).
        The function expects the YAML schema you have been using
        (`start_date`, `end_date`, `portfolio_input`, `stock_factor_proxies`, …).

    Side-effects
    ------------
    • Prints a formatted risk report to stdout.
    • Does **not** return anything; everything is handled inline.
      (If you need the raw DataFrames, simply return `summary`, `df_risk`,
      and `df_beta` at the end.)

    Example
    -------
    >>> run_portfolio("portfolio.yaml")
    === Target Allocations ===
    …                                 # summary table
    === Portfolio Risk Limit Checks ===
    Volatility:             21.65%  ≤ 40.00%     → PASS
    …
    === Beta Exposure Checks ===
    market       β = 0.74  ≤ 0.80  → PASS
    …
    """
    import yaml
    import pandas as pd
    from run_portfolio_risk import (
        standardize_portfolio_input,
        latest_price,
        display_portfolio_summary,
        evaluate_portfolio_beta_limits,
        evaluate_portfolio_risk_limits,
    )
    from portfolio_risk import build_portfolio_view
    from risk_helpers import (
        get_worst_monthly_factor_losses,
        aggregate_worst_losses_by_factor_type
    )

    # ─── 1. Load YAML Inputs ─────────────────────────────────
    with open(filepath, "r") as f:
        config = yaml.safe_load(f)
    with open("risk_limits.yaml", "r") as f:
        risk_config = yaml.safe_load(f)

    weights = standardize_portfolio_input(config["portfolio_input"], latest_price)["weights"]
    summary = build_portfolio_view(
        weights,
        config["start_date"],
        config["end_date"],
        config.get("expected_returns"),
        config.get("stock_factor_proxies")
    )

    # ─── 2. Display Summary ─────────────────────────────────
    display_portfolio_summary(summary)

    # ─── 3. Compute Beta Limits ─────────────────────────────
    start_date = config["start_date"]
    end_date   = config["end_date"]
    proxies    = config["stock_factor_proxies"]
    loss_limit = risk_config["max_single_factor_loss"]

    worst_losses = get_worst_monthly_factor_losses(proxies, start_date, end_date)
    worst_by_type = aggregate_worst_losses_by_factor_type(proxies, worst_losses)

    max_betas = {
        factor: float('inf') if loss >= 0 else loss_limit / loss
        for factor, (_, loss) in worst_by_type.items()
    }

    # ─── 4. Evaluate Portfolio Risk Rules ───────────────────
    print("\n=== Portfolio Risk Limit Checks ===")
    df_risk = evaluate_portfolio_risk_limits(
        summary,
        risk_config["portfolio_limits"],
        risk_config["concentration_limits"],
        risk_config["variance_limits"]
    )
    for _, row in df_risk.iterrows():
        status = "→ PASS" if row["Pass"] else "→ FAIL"
        print(f"{row['Metric']:<22} {row['Actual']:.2%}  ≤ {row['Limit']:.2%}  {status}")

    # ─── 5. Evaluate Beta Limits ────────────────────────────
    print("\n=== Beta Exposure Checks ===")
    df_beta = evaluate_portfolio_beta_limits(
        summary["portfolio_factor_betas"],
        max_betas
    )
    
    for factor, row in df_beta.iterrows():
        status = "→ PASS" if row["Pass"] else "→ FAIL"      # ← capital P
        print(f"{factor:<12} β = {row['Beta']:.2f}  ≤ {row['Max Beta']:.2f}  {status}")

In [ ]:
# ─── risk_helpers.py ─────────────────────────────────────────────────────────

from __future__ import annotations
from typing import Dict, Tuple, List
from datetime import datetime
import yaml
import pandas as pd

def calc_max_factor_betas(
    portfolio_yaml: str = "portfolio.yaml",
    risk_yaml: str = "risk_limits.yaml",
    lookback_years: int = 10,
    echo: bool = True,
) -> Dict[str, float]:
    """
    Derive max-allowable portfolio betas for each factor type from
    historical worst 1-month factor proxy returns.

    Parameters
    ----------
    portfolio_yaml : str
        Path to the YAML file containing `stock_factor_proxies`.
    risk_yaml : str
        Path to YAML containing `max_single_factor_loss`.
    lookback_years : int
        Historical window length to scan (ending today).
    echo : bool
        If True, pretty-prints the intermediate tables to stdout.

    Returns
    -------
    Dict[str, float]
        {factor_type: max_beta}.  Example:
        {'market': 0.67, 'momentum': 0.79, ...}
    """
    # 1. --- load configs -----------------------------------------------------
    with open(portfolio_yaml, "r") as f:
        port_cfg = yaml.safe_load(f)
    with open(risk_yaml, "r") as f:
        risk_cfg = yaml.safe_load(f)

    proxies = port_cfg["stock_factor_proxies"]
    loss_limit = risk_cfg["max_single_factor_loss"]  # e.g. -0.10

    # 2. --- date window ------------------------------------------------------
    end_dt = datetime.today()
    start_dt = end_dt - pd.DateOffset(years=lookback_years)
    end_str, start_str = end_dt.strftime("%Y-%m-%d"), start_dt.strftime("%Y-%m-%d")

    # 3. --- worst per-proxy --------------------------------------------------
    worst_per_proxy = get_worst_monthly_factor_losses(
        proxies, start_str, end_str
    )

    # 4. --- worst per factor-type -------------------------------------------
    worst_by_factor = aggregate_worst_losses_by_factor_type(
        proxies, worst_per_proxy
    )

    # 5. --- max beta calc ----------------------------------------------------
    max_betas = {
        ftype: float("inf") if worst >= 0 else loss_limit / worst
        for ftype, (_, worst) in worst_by_factor.items()
    }

    # --- pretty print block --------------------------------------------------
    if echo:
        print("=== Worst Monthly Losses per Proxy ===")
        for p, v in sorted(worst_per_proxy.items(), key=lambda kv: kv[1]):
            print(f"{p:<12} : {v:.2%}")

        print("\n=== Worst Monthly Losses per Factor Type ===")
        for ftype, (p, v) in worst_by_factor.items():
            print(f"{ftype:<10} → {p:<12} : {v:.2%}")

        print(f"\n=== Max Allowable Beta per Factor "
              f"(Loss Limit = {loss_limit:.0%}) ===")
        for ftype, beta in max_betas.items():
            print(f"{ftype:<10} → β ≤ {beta:.2f}")

    return max_betas

In [ ]:
# === What-If Risk Calculations with New Weights ===

# assuming `weights`, `risk_config`, `config`, `stock_factor_proxies` in scope

delta = {"TW": 0.05, "PCTY": -0.02}      # +5 ppts TW, −2 ppts PCTY
summary_what, risk_what, beta_what = run_what_if(
    weights, delta,
    risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies
)

In [ ]:
# === Optimizer: Minimum Variance Weights ===

new_w, risk_tbl, beta_tbl = run_min_var_optimiser(
    weights,
    risk_config,
    config["start_date"],
    config["end_date"],
    stock_factor_proxies,
    echo=True,          # turn off to suppress prints
)

In [ ]:
# === What-If Risk Calculations with New Weights & Before / After Risk ===

# 1) What-if run --------------------------------------------------
delta = {"TW": +0.05, "PCTY": -0.02}     # +5 ppts TW, −2 ppts PCTY

summary_new, risk_new, beta_new = run_what_if(
    weights,
    delta,
    risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies
)

# --- 2) Baseline (unchanged portfolio) -------------------------------
risk_base, beta_base = evaluate_weights(
    weights, risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies
)

# --- 3) Diff & pretty-print -----------------------------------------
cmp_risk = (compare_risk_tables(risk_base, risk_new)
              .set_index("Metric")
              .loc[risk_base["Metric"].tolist()]        # keep original row order
              .reset_index()
           )

cmp_beta = (compare_beta_tables(beta_base, beta_new)
                .reindex(beta_base.index)  
           )
            
print("\n📐  Risk Limits — Before vs After\n")
print(cmp_risk.to_string(index=False,
                         formatters={"Old": _fmt_pct, "New": _fmt_pct,
                                     "Δ": _fmt_pct, "Limit": _fmt_pct}))

print("\n📊  Factor Betas — Before vs After\n")
print(
    cmp_beta.to_string(
        formatters={
            "Old":       "{:.2f}".format,   # two-decimals, e.g. 0.5830
            "New":       "{:.2f}".format,
            "Δ":         "{:.2f}".format,
            "Max Beta":  "{:.2f}".format,   # keep two-decimals on Max Beta if you like
            "Old Pass":  lambda x: "PASS" if x else "FAIL",
            "New Pass":  lambda x: "PASS" if x else "FAIL",
        }
    )
)

In [ ]:
# File: run_portfolio_risk.py

import pandas as pd
from typing import Dict

def evaluate_portfolio_beta_limits(
    portfolio_factor_betas: pd.Series,
    max_betas: Dict[str, float]
) -> pd.DataFrame:
    """
    Compares each factor's actual portfolio beta to the allowable max beta.

    Args:
        portfolio_factor_betas (pd.Series): e.g. {"market": 0.74, "momentum": 1.1, ...}
        max_betas (Dict[str, float]): e.g. {"market": 0.80, "momentum": 1.56, ...}

    Returns:
        pd.DataFrame: Table with actual vs. max beta and pass/fail status.
    """
    data = []
    for factor, max_b in max_betas.items():
        actual = portfolio_factor_betas.get(factor, 0.0)
        result = {
            "factor": factor,
            "portfolio_beta": actual,
            "max_allowed_beta": max_b,
            "pass": abs(actual) <= max_b,
            "buffer": max_b - abs(actual)
        }
        data.append(result)

    df = pd.DataFrame(data).set_index("factor")
    return df[["portfolio_beta", "max_allowed_beta", "pass", "buffer"]]

In [ ]:
# File: risk_runner.py
# Check portfolio beta limit checks 

df_beta_check = evaluate_portfolio_beta_limits(
    summary["portfolio_factor_betas"],
    max_betas
)
print("=== Portfolio Exposure Limit Checks ===\n")
print(df_beta_check)

In [ ]:
# File: risk_runner.py
# Check portfolio beta limit checks

df_beta_check = evaluate_portfolio_beta_limits(
    portfolio_factor_betas = summary["portfolio_factor_betas"],
    max_betas              = max_betas,
    proxy_betas            = summary["industry_variance"].get("per_industry_group_beta"),
    max_proxy_betas        = max_betas_by_proxy
)

print("=== Portfolio Exposure Limit Checks ===\n")
print(df_beta_check.to_string(formatters={
    "portfolio_beta":   "{:.2f}".format,
    "max_allowed_beta": "{:.2f}".format,
    "buffer":           "{:.2f}".format,
    "pass":             lambda x: "PASS" if x else "FAIL"
}))

In [ ]:
# ─── 2b. Top Factor Variance Contributors ───────────────
wfv = summary["weighted_factor_var"]
total_var = summary["variance_decomposition"]["portfolio_variance"]

top_contrib = (
    wfv.stack()
       .sort_values(ascending=False)
       .head(10)
       .rename_axis(index=["Ticker", "Factor"])
       .reset_index(name="w²·β²·σ²")
)

top_contrib["% of Total Var"] = top_contrib["w²·β²·σ²"] / total_var

print("\n=== Top 10 Factor Variance Contributors (stock × factor) ===")
print(top_contrib.to_string(index=False, formatters={
    "w²·β²·σ²": "{:.6f}".format,
    "% of Total Var": "{:.1%}".format
}))

In [ ]:
# File: portfolio_optimizer.py

# ---------------------------------------------------------------------
# Max-return subject to risk limits
# ---------------------------------------------------------------------

import cvxpy as cp
import numpy as np
import pandas as pd
from typing import Dict, Union, List

def solve_max_return_with_risk_limits(
    init_weights: Dict[str, float],
    risk_cfg: Dict[str, Dict[str, float]],
    start_date: str,
    end_date: str,
    stock_factor_proxies: Dict[str, Dict[str, Union[str, List[str]]]],
    expected_returns: Dict[str, float],
    allow_short: bool = False
) -> Dict[str, float]:
    """
    Max-return portfolio subject to firm-wide limits.

    Objective
    ---------
    Maximise Σ w_i·μ_i   (μ = expected annual return in decimals)
    subject to ALL risk limits defined in `risk_cfg`.

    Risk Constraints
    -----------
    1. Σ wᵢ = 1                               (fully invested)  
    2. wᵢ ≥ 0  if ``allow_short`` is False    (long-only)  
    3. wᵢ ≤ single-name cap                   (concentration limit)  
    4. √(12 · wᵀΣw) ≤ σ_cap                   (annual vol cap)  
       – Σ is the monthly covariance from *start_date*–*end_date*.

    This is a convex quadratic program solved with CVXPY + ECOS.

    Parameters
    ----------
    init_weights          : current {ticker: weight}.
    risk_cfg              : parsed risk_limits.yaml (needs the three sub-dicts).
    start_date / end_date : window used to build covariance & factor tables.
    stock_factor_proxies  : same structure used elsewhere in your toolkit.
    expected_returns      : {ticker: annual exp return}.  **Required**.
    allow_short           : set True if negatives are allowed.

    Returns
    -------
    dict {ticker: new_weight}.  Sums to 1 by construction.
    """
    # ---- 1. Build Σ (monthly) with your existing helper ----------------
    from portfolio_risk import build_portfolio_view   # re-use, keeps code DRY

    tickers = list(init_weights.keys())
    view = build_portfolio_view(
        init_weights, start_date, end_date,
        expected_returns=None,           # we only need cov matrix
        stock_factor_proxies=stock_factor_proxies
    )
    Σ_m = view["covariance_matrix"].loc[tickers, tickers].values
    μ    = np.array([expected_returns.get(t, 0.0) for t in tickers])

    if np.allclose(μ, 0):
        raise ValueError("expected_returns is empty or zeros – nothing to maximise")

    # ---- 2. CVXPY variables & objective --------------------------------
    w = cp.Variable(len(tickers))
    objective = cp.Maximize(μ @ w)     # linear objective

    # ---- 3. Constraints -------------------------------------------------
    cons = []

    # fully invested
    cons += [cp.sum(w) == 1]

    # long-only?
    if not allow_short:
        cons += [w >= 0]

    # single-name cap
    w_cap = risk_cfg["concentration_limits"]["max_single_stock_weight"]
    cons += [w <= w_cap]

    # volatility cap  (monthly Σ → annual σ = √12·sqrt(wᵀΣw))
    σ_cap = risk_cfg["portfolio_limits"]["max_volatility"]
    cons += [cp.sqrt(cp.quad_form(w, Σ_m)) * np.sqrt(12) <= σ_cap]

    # You can add more factor / industry caps here if you want them
    # (reuse values already in risk_cfg).

    # ---- 4. Solve -------------------------------------------------------
    prob = cp.Problem(objective, cons)
    prob.solve(solver=cp.ECOS, qcp=True, verbose=False)

    if prob.status not in ("optimal", "optimal_inaccurate"):
        raise ValueError(f"Solver returned status {prob.status}")

    return {t: float(v) for t, v in zip(tickers, w.value)}

In [ ]:
# === Optimizer: Highest Return Weights within Risk Limits ===

# 1. run optimiser → weights only
w_opt = solve_max_return_with_risk_limits(
    weights,                    # current allocation
    risk_config,                # parsed risk_limits.yaml
    config["start_date"],
    config["end_date"],
    stock_factor_proxies,
    expected_returns=config["expected_returns"]  # MUST be present
)

print("\n🎯  Target max-return, risk-constrained weights:\n")
for k,v in sorted(w_opt.items(), key=lambda kv: -kv[1]):
    if abs(v) > 1e-4:
        print(f"{k:<8} : {v:.2%}")

# 2. whenever you want: evaluate & pretty-print
risk_tbl, beta_tbl = evaluate_weights(
    w_opt, risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies
)

print("\n📐  Max-return Portfolio – Risk Checks\n")
pct = lambda x: f"{x:.2%}"
print(risk_tbl.to_string(index=False, formatters={"Actual": pct, "Limit": pct}))

print("\n📊  Max-return Portfolio – Factor Betas\n")
beta_tbl = _drop_factors(beta_tbl)
print(beta_tbl.to_string(formatters={
    "Beta": "{:.2f}".format, "Max Beta": "{:.2f}".format,
    "Buffer": "{:.2f}".format, "Pass": lambda x: "PASS" if x else "FAIL"
}))

# 3)  Aggregate-vs-Industry exposure checks
from portfolio_risk          import build_portfolio_view
from run_portfolio_risk      import evaluate_portfolio_beta_limits

# rebuild a summary on the optimised weights so we can grab per-industry betas
summary_opt = build_portfolio_view(
    w_opt,
    config["start_date"], config["end_date"],
    expected_returns=None,
    stock_factor_proxies=stock_factor_proxies
)

df_beta_chk = evaluate_portfolio_beta_limits(
    portfolio_factor_betas = summary_opt["portfolio_factor_betas"],
    max_betas              = max_betas,                       # already computed earlier
    proxy_betas            = summary_opt["industry_variance"]["per_industry_group_beta"],
    max_proxy_betas        = max_betas_by_proxy               # already computed earlier
)

df_factors = df_beta_chk[~df_beta_chk.index.str.startswith("industry_proxy::")]
df_proxies = (df_beta_chk
              .loc[df_beta_chk.index.str.startswith("industry_proxy::")]
              .copy())
df_proxies.index = df_proxies.index.str.replace("industry_proxy::", "")

fmt = {
    "portfolio_beta":   "{:+.2f}".format,
    "max_allowed_beta": "{:.2f}".format,
    "buffer":           "{:+.2f}".format,
    "pass":             lambda x: "PASS" if x else "FAIL"
}

print("\n=== Industry Exposure Checks ===\n")
print(df_proxies.to_string(index_names=False, formatters=fmt))

In [ ]:
# === Optimizer: Highest Return Weights within Risk Limits ===

# 1. run optimiser → weights only
w_opt = solve_max_return_with_risk_limits(
    weights,                    # current allocation
    risk_config,                # parsed risk_limits.yaml
    config["start_date"],
    config["end_date"],
    stock_factor_proxies,
    expected_returns=config["expected_returns"]  # MUST be present
)

print("\n🎯  Target max-return, risk-constrained weights:\n")
for k,v in sorted(w_opt.items(), key=lambda kv: -kv[1]):
    if abs(v) > 1e-4:
        print(f"{k:<8} : {v:.2%}")


# 2) main risk / factor tables (aggregate betas only)
risk_tbl, beta_tbl = evaluate_weights(
    w_opt, risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies,
)

# 3) industry-proxy table
summary_opt = build_portfolio_view(
    w_opt, config["start_date"], config["end_date"],
    expected_returns=None,
    stock_factor_proxies=stock_factor_proxies
)

df_beta_chk = evaluate_portfolio_beta_limits(
    portfolio_factor_betas = summary_opt["portfolio_factor_betas"],
    max_betas              = max_betas,   # already computed earlier
    proxy_betas            = summary_opt["industry_variance"]["per_industry_group_beta"],
    max_proxy_betas        = max_betas_by_proxy,
)

# --- pretty-print ---------------------------------------------------------
fmt = {
    "portfolio_beta":   "{:+.2f}".format,
    "max_allowed_beta": "{:.2f}".format,
    "buffer":           "{:+.2f}".format,
    "pass":             lambda x: "PASS" if x else "FAIL",
}

df_factors  = df_beta_chk[~df_beta_chk.index.str.startswith("industry_proxy::")]
df_proxies  = df_beta_chk[ df_beta_chk.index.str.startswith("industry_proxy::")].copy()
df_proxies.index = df_proxies.index.str.replace("industry_proxy::", "")

print("\n📐  Max-return Portfolio – Risk Checks\n")
pct = lambda x: f"{x:.2%}"
print(risk_tbl.to_string(index=False, formatters={"Actual": pct, "Limit": pct}))

print("\n📊  Aggregate Factor Exposures\n")
print(df_factors.to_string(index_names=False, formatters=fmt))

print("\n📊  Industry Exposure Checks\n")
print(df_proxies.to_string(index_names=False, formatters=fmt))

In [ ]:
# === What-If Risk Calculations using Inputs for New Weights with Before / After Risk ===

# ──────────────────────────────────────────────────────────────────────────────
# WHAT-IF DRIVER
#
# Input precedence
# ----------------
# 1. If `what_if_portfolio.yaml` contains a top-level `new_weights:` section
#    → treat as a full-replacement portfolio.
#      • `shift_dict` is ignored in this case.
#
# 2. Otherwise, build an incremental *delta* dict:
#      • YAML `delta:` values are parsed first.
#      • Any overlapping keys in `shift_dict` overwrite the YAML values.
#
# 3. Branch logic
#      • full-replacement  → evaluate_weights(new_weights_yaml)
#      • incremental tweak → run_what_if(base_weights, delta)
#
# 4. After computing the new portfolio’s risk/beta tables once,
#    we also compute the baseline (unchanged) tables once, then
#    show before-vs-after diffs.
#
# Note: No function ever writes back to the YAML file; all merges happen
#       in memory.
# ──────────────────────────────────────────────────────────────────────────────

# 1) Parse input ────────────────────────────────────────────────
delta, new_weights_yaml = parse_delta(
    yaml_path="what_if_portfolio.yaml",      # or None
    literal_shift=shift_dict       # None or {"TW": "+500bp", …}
)

# 2) Build the *new* portfolio once ─────────────────────────────
if new_weights_yaml:                # full-replacement supplied in YAML
    risk_new, beta_new = evaluate_weights(
        new_weights_yaml, risk_config,
        config["start_date"], config["end_date"],
        stock_factor_proxies
    )
    _print_single_portfolio(risk_new, beta_new, title="New Portfolio What-if")   
    
else:                              # incremental tweak path (delta shift)
    summary_new, risk_new, beta_new = run_what_if(
        base_weights=weights,
        delta=delta,
        risk_cfg=risk_config,
        start_date=config["start_date"],
        end_date=config["end_date"],
        factor_proxies=stock_factor_proxies
    )

# 3) Baseline portfolio (unchanged) ────────────────────────────
risk_base, beta_base = evaluate_weights(
    weights, risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies
)

# 4) Diff & pretty-print ───────────────────────────────────────
cmp_risk = (compare_risk_tables(risk_base, risk_new)
              .set_index("Metric")
              .loc[risk_base["Metric"].tolist()]        # keep original row order
              .reset_index()
           )

cmp_beta = (compare_beta_tables(beta_base, beta_new)
                .reindex(beta_base.index)  
           )

cmp_beta = _drop_factors(cmp_beta)

print("\n📐  Risk Limits — Before vs After\n")
print(cmp_risk.to_string(index=False,
                         formatters={"Old": _fmt_pct, "New": _fmt_pct,
                                     "Δ": _fmt_pct, "Limit": _fmt_pct}))

print("\n📊  Factor Betas — Before vs After\n")
print(
    cmp_beta.to_string(
        formatters={
            "Old":       "{:.2f}".format,   # two-decimals, e.g. 0.5830
            "New":       "{:.2f}".format,
            "Δ":         "{:.2f}".format,
            "Max Beta":  "{:.2f}".format,   # keep two-decimals on Max Beta if you like
            "Old Pass":  lambda x: "PASS" if x else "FAIL",
            "New Pass":  lambda x: "PASS" if x else "FAIL",
        }
    )
)

In [ ]:
# Add right after you build df_proxies_new

# --- baseline proxy betas -----------------------------------------
df_beta_chk_base = evaluate_portfolio_beta_limits(
    portfolio_factor_betas = summary_base["portfolio_factor_betas"],
    max_betas              = max_betas,
    proxy_betas            = summary_base["industry_variance"]["per_industry_group_beta"],
    max_proxy_betas        = max_betas_by_proxy,
).loc[df_proxies_new.index]        # align rows to new

cmp_proxy = pd.DataFrame({
    "Old":    df_beta_chk_base["portfolio_beta"],
    "New":    df_proxies_new["portfolio_beta"],
    "Δ":      df_proxies_new["portfolio_beta"] - df_beta_chk_base["portfolio_beta"],
    "Max Beta": df_proxies_new["max_allowed_beta"],
    "Old Pass": df_beta_chk_base["pass"],
    "New Pass": df_proxies_new["pass"],
})

print("\n📊  Industry Betas — Before vs After\n")
print(cmp_proxy.to_string(formatters=_fmt_beta))

In [ ]:
# === What-If Risk Calculations using Inputs for New Weights with Before / After Risk ===

# ──────────────────────────────────────────────────────────────────────────────
# WHAT-IF DRIVER
#
# Input precedence
# ----------------
# 1. If `what_if_portfolio.yaml` contains a top-level `new_weights:` section
#    → treat as a full-replacement portfolio.
#      • `shift_dict` is ignored in this case.
#
# 2. Otherwise, build an incremental *delta* dict:
#      • YAML `delta:` values are parsed first.
#      • Any overlapping keys in `shift_dict` overwrite the YAML values.
#
# 3. Branch logic
#      • full-replacement  → evaluate_weights(new_weights_yaml)
#      • incremental tweak → run_what_if(base_weights, delta)
#
# 4. After computing the new portfolio’s risk/beta tables once,
#    we also compute the baseline (unchanged) tables once, then
#    show before-vs-after diffs.
#
# Note: No function ever writes back to the YAML file; all merges happen
#       in memory.
# ──────────────────────────────────────────────────────────────────────────────

from run_portfolio_risk import evaluate_portfolio_beta_limits, evaluate_portfolio_risk_limits
from risk_helpers       import compute_max_betas
from portfolio_risk     import build_portfolio_view

# ------------------------------------------------------------------ 1) Parse
delta, new_weights_yaml = parse_delta(
    yaml_path="what_if_portfolio.yaml",
    literal_shift=shift_dict,
)

# ------------------------------------------------------------------ 2) NEW portfolio
if new_weights_yaml:                 # full-replacement path
    risk_new, beta_new = evaluate_weights(
        new_weights_yaml, risk_config,
        config["start_date"], config["end_date"],
        stock_factor_proxies,
    )
    summary_new = build_portfolio_view(
        new_weights_yaml,
        config["start_date"], config["end_date"],
        expected_returns=None,
        stock_factor_proxies=stock_factor_proxies,
    )
    _print_single_portfolio(risk_new, beta_new, title="New Portfolio What-if")

else:                                # delta-shift path
    summary_new, risk_new, beta_new = run_what_if(
        base_weights   = weights,
        delta          = delta,
        risk_cfg       = risk_config,
        start_date     = config["start_date"],
        end_date       = config["end_date"],
        factor_proxies = stock_factor_proxies,
    )

# ---------------------------------------------------------------- 2-b) β tables (factor + proxy) – NEW only
max_betas = compute_max_betas(             # <-- single return value
    stock_factor_proxies,
    config["start_date"], config["end_date"],
    loss_limit_pct = risk_config["max_single_factor_loss"],
)

# `max_betas_by_proxy` was already computed earlier via `calc_max_factor_betas`.
# If that cell hasn’t been executed yet, run it once before this block.

df_beta_chk_new = evaluate_portfolio_beta_limits(
    portfolio_factor_betas = summary_new["portfolio_factor_betas"],
    max_betas              = max_betas,
    proxy_betas            = summary_new["industry_variance"]["per_industry_group_beta"],
    max_proxy_betas        = max_betas_by_proxy,
)

df_factors_new = df_beta_chk_new[~df_beta_chk_new.index.str.startswith("industry_proxy::")]
df_proxies_new = df_beta_chk_new[ df_beta_chk_new.index.str.startswith("industry_proxy::")].copy()
df_proxies_new.index = df_proxies_new.index.str.replace("industry_proxy::", "")

_fmt_beta = {
    "portfolio_beta":   "{:+.2f}".format,
    "max_allowed_beta": "{:.2f}".format,
    "buffer":           "{:+.2f}".format,
    "pass":             lambda x: "PASS" if x else "FAIL",
}

print("\n📊  Industry Exposure Checks\n")
print(df_proxies_new.to_string(index_names=False, formatters=_fmt_beta))

# ------------------------------------------------------------------ 3) BASELINE portfolio
risk_base, beta_base = evaluate_weights(
    weights, risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies,
)

# ------------------------------------------------------------------ 4) DIFF & pretty-print
cmp_risk = (
    compare_risk_tables(risk_base, risk_new)
    .set_index("Metric")
    .loc[risk_base["Metric"].tolist()]       # keep original row order
    .reset_index()
)

cmp_beta = (
    compare_beta_tables(beta_base, beta_new)
    .reindex(beta_base.index)
)
cmp_beta = _drop_factors(cmp_beta)

print("\n📐  Risk Limits — Before vs After\n")
print(
    cmp_risk.to_string(
        index=False,
        formatters={
            "Old": _fmt_pct,
            "New": _fmt_pct,
            "Δ":   _fmt_pct,
            "Limit": _fmt_pct,
        },
    )
)

print("\n📊  Factor Betas — Before vs After\n")
print(
    cmp_beta.to_string(
        formatters={
            "Old":       "{:.2f}".format,
            "New":       "{:.2f}".format,
            "Δ":         "{:.2f}".format,
            "Max Beta":  "{:.2f}".format,
            "Old Pass":  lambda x: "PASS" if x else "FAIL",
            "New Pass":  lambda x: "PASS" if x else "FAIL",
        }
    )
)

In [ ]:
# === What-If Risk Calculations using Shift Inputs ===

shift_dict = {"TW": "+500bp", "PCTY": "-200bp"}

In [ ]:
# === What-If Risk Calculations (Before / After) ============================

# ──────────────────────────────────────────────────────────────────────────────
# WHAT-IF DRIVER
#
# Input precedence
# ----------------
# 1. If `what_if_portfolio.yaml` contains a top-level `new_weights:` section
#    → treat as a full-replacement portfolio.
#      • `shift_dict` is ignored in this case.
#
# 2. Otherwise, build an incremental *delta* dict:
#      • YAML `delta:` values are parsed first.
#      • Any overlapping keys in `shift_dict` overwrite the YAML values.
#
# 3. Branch logic
#      • full-replacement  → evaluate_weights(new_weights_yaml)
#      • incremental tweak → run_what_if(base_weights, delta)
#
# 4. After computing the new portfolio’s risk/beta tables once,
#    we also compute the baseline (unchanged) tables once, then
#    show before-vs-after diffs.
#
# Note: No function ever writes back to the YAML file; all merges happen
#       in memory.
# ──────────────────────────────────────────────────────────────────────────────

from run_portfolio_risk import (
    evaluate_portfolio_risk_limits,
    evaluate_portfolio_beta_limits,
)
from risk_helpers   import compute_max_betas
from portfolio_risk import build_portfolio_view

_FMT_PCT  = lambda x: f"{x:.1%}"
_FMT_BETA = {
    "portfolio_beta":   "{:.2f}".format,
    "max_allowed_beta": "{:.2f}".format,
    "buffer":           "{:.2f}".format,
    "pass":             lambda x: "PASS" if x else "FAIL",
}

# ── 0) make sure we have per-proxy β caps in memory ------------------------
try:
    max_betas_by_proxy                           # noqa: F821 if not yet defined
except NameError:
    # run once, silent
    _, max_betas_by_proxy = calc_max_factor_betas(
        portfolio_yaml = "portfolio.yaml",
        risk_yaml      = "risk_limits.yaml",
        lookback_years = 10,
        echo           = False,      # ← no console spam
    )

# ── 1) Parse scenario ------------------------------------------------------
delta, new_weights_yaml = parse_delta(
    yaml_path   = "what_if_portfolio.yaml",   # ↙ or None
    literal_shift = shift_dict,               # ↙ or None
)

# ── 2) Build NEW summary / tables -----------------------------------------
if new_weights_yaml:     
    # full-replacement branch
    # --- guard: re-normalise just in case the YAML doesn’t sum to 1.0 ----------
    new_weights_yaml = normalize_weights(new_weights_yaml)
    summary_new = build_portfolio_view(
        new_weights_yaml,
        config["start_date"], config["end_date"],
        expected_returns=None,
        stock_factor_proxies=stock_factor_proxies,
    )
else:
    # incremental tweaks
    summary_new, *_ = run_what_if(
        base_weights   = weights,
        delta          = delta,
        risk_cfg       = risk_config,
        start_date     = config["start_date"],
        end_date       = config["end_date"],
        factor_proxies = stock_factor_proxies,
        verbose        = False,   
    )

# Risk table (NEW)
risk_new = evaluate_portfolio_risk_limits(
    summary_new,
    risk_config["portfolio_limits"],
    risk_config["concentration_limits"],
    risk_config["variance_limits"],
)

# β caps
max_betas = compute_max_betas(
    stock_factor_proxies,
    config["start_date"], config["end_date"],
    loss_limit_pct = risk_config["max_single_factor_loss"],
)
# max_betas_by_proxy was produced earlier by calc_max_factor_betas
beta_new = evaluate_portfolio_beta_limits(
    summary_new["portfolio_factor_betas"],
    max_betas,
    proxy_betas     = summary_new["industry_variance"]["per_industry_group_beta"],
    max_proxy_betas = max_betas_by_proxy,
)

# Split β table → factors / proxies
beta_f_new = beta_new[~beta_new.index.str.startswith("industry_proxy::")]
beta_p_new = beta_new[  beta_new.index.str.startswith("industry_proxy::")].copy()
beta_p_new.index = beta_p_new.index.str.replace("industry_proxy::", "")

# ── 3) Baseline (unchanged) -----------------------------------------------
summary_base = build_portfolio_view(
    weights,
    config["start_date"], config["end_date"],
    expected_returns=None,
    stock_factor_proxies=stock_factor_proxies,
)
risk_base = evaluate_portfolio_risk_limits(
    summary_base,
    risk_config["portfolio_limits"],
    risk_config["concentration_limits"],
    risk_config["variance_limits"],
)
beta_base = evaluate_portfolio_beta_limits(
    summary_base["portfolio_factor_betas"],
    max_betas,
    proxy_betas     = summary_base["industry_variance"]["per_industry_group_beta"],
    max_proxy_betas = max_betas_by_proxy,
)

# ── 4) Diffs ---------------------------------------------------------------
cmp_risk  = compare_risk_tables(risk_base,  risk_new)
cmp_beta  = compare_beta_tables(beta_base,  beta_new)
cmp_beta  = _drop_factors(cmp_beta)         # suppress “industry” agg row
cmp_beta  = cmp_beta[~cmp_beta.index.str.startswith("industry_proxy::")] # suppress proxy rows
 
# after you build cmp_risk - match table order
order = risk_new["Metric"].tolist()          # or risk_tbl if that is the one you printed
cmp_risk = (
    cmp_risk
    .set_index("Metric")                     # make 'Metric' the index
    .loc[order]                              # re-order rows
    .reset_index()                           # back to column form
)

# ── 5) Pretty-print --------------------------------------------------------
print("\n📐  NEW Portfolio – Risk Checks\n")
print(risk_new.to_string(index=False, formatters={"Actual": _FMT_PCT, "Limit": _FMT_PCT}))

print("\n📊  NEW Aggregate Factor Exposures\n")
print(beta_f_new.to_string(index_names=False, formatters=_FMT_BETA))

print("\n📊  NEW Industry Exposure Checks\n")
print(beta_p_new.to_string(index_names=False, formatters=_FMT_BETA))

print("\n📐  Risk Limits — Before vs After\n")
print(cmp_risk.to_string(index=False, formatters={"Old": _FMT_PCT, "New": _FMT_PCT,
                                                  "Δ": _FMT_PCT,  "Limit": _FMT_PCT}))

print("\n📊  Factor Betas — Before vs After\n")
print(
    cmp_beta.to_string(
        index_names=False,          # ← hide the “factor” header
        formatters={
    "Old":       "{:.2f}".format,
    "New":       "{:.2f}".format,
    "Δ":         "{:.2f}".format,
    "Max Beta":  "{:.2f}".format,
    "Old Pass":  lambda x: "PASS" if x else "FAIL",
    "New Pass":  lambda x: "PASS" if x else "FAIL",
}))

In [ ]:
# === Optimizer: Lowest Variance Weights within Risk Limits ===

# 1. get the weights
w_min = run_min_var_optimiser(
    weights,
    risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies,
    echo=True          # or False if you don’t want the quick print-out
)

# 2. when you actually need the risk tables ↓
risk_tbl, beta_tbl = evaluate_weights(
    w_min, risk_config,
    config["start_date"], config["end_date"],
    stock_factor_proxies
)

# 3a. risk table
print("\n📐  Optimised Portfolio – Risk Checks\n")
pct = lambda x: f"{x:.2%}"
print(risk_tbl.to_string(index=False,
                        formatters={"Actual": pct, "Limit": pct}))

# 3b. beta table
print("\n📊  Optimised Portfolio – Factor Betas\n")
beta_tbl = _drop_factors(beta_tbl)
print(beta_tbl.to_string(formatters={
    "Beta":      "{:.2f}".format,
    "Max Beta":  "{:.2f}".format,
    "Buffer":    "{:.2f}".format,
    "Pass":      lambda x: "PASS" if x else "FAIL",
}))
# …then display / log those tables however you like

In [ ]:
# === Optimizer: Highest Return Weights within Risk Limits ===

# 1. run optimiser → weights only
w_opt = solve_max_return_with_risk_limits(
    weights,                    # current allocation
    risk_config,                # parsed risk_limits.yaml
    config["start_date"],
    config["end_date"],
    stock_factor_proxies,
    expected_returns=config["expected_returns"]  # MUST be present
)

print("\n🎯  Target max-return, risk-constrained weights:\n")
for k,v in sorted(w_opt.items(), key=lambda kv: -kv[1]):
    if abs(v) > 1e-4:
        print(f"{k:<8} : {v:.1%}")

# 2) main risk / factor tables

# single summary build
summary_opt = build_portfolio_view(
    w_opt, config["start_date"], config["end_date"],
    expected_returns=None,
    stock_factor_proxies=stock_factor_proxies,
)

# risk table
risk_tbl = evaluate_portfolio_risk_limits(
    summary_opt,
    risk_config["portfolio_limits"],
    risk_config["concentration_limits"],
    risk_config["variance_limits"],
)

# beta tables (aggregate + proxy)
beta_tbl_agg = evaluate_portfolio_beta_limits(
    summary_opt["portfolio_factor_betas"],
    max_betas,           # factor caps
)

df_beta_chk = evaluate_portfolio_beta_limits(
    summary_opt["portfolio_factor_betas"],
    max_betas,
    proxy_betas     = summary_opt["industry_variance"]["per_industry_group_beta"],
    max_proxy_betas = max_betas_by_proxy,
)

# --- pretty-print ---------------------------------------------------------
fmt = {
    "portfolio_beta":   "{:.2f}".format,
    "max_allowed_beta": "{:.2f}".format,
    "buffer":           "{:.2f}".format,
    "pass":             lambda x: "PASS" if x else "FAIL",
}

df_factors  = df_beta_chk[~df_beta_chk.index.str.startswith("industry_proxy::")]
df_proxies  = df_beta_chk[ df_beta_chk.index.str.startswith("industry_proxy::")].copy()
df_proxies.index = df_proxies.index.str.replace("industry_proxy::", "")

print("\n📐  Max-return Portfolio – Risk Checks\n")
pct = lambda x: f"{x:.2%}"
print(risk_tbl.to_string(index=False, formatters={"Actual": pct, "Limit": pct}))

print("\n📊  Aggregate Factor Exposures\n")
print(df_factors.to_string(index_names=False, formatters=fmt))

print("\n📊  Industry Exposure Checks\n")
print(df_proxies.to_string(index_names=False, formatters=fmt))

In [ ]:
# File: data_loader.py

import requests
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime
from typing import Optional, Union, List, Dict

# Configuration
API_KEY  = "0ZDOD7zoxCPQLDOyDw5e1tE8bwEFfKWk"
BASE_URL = "https://financialmodelingprep.com/stable"


def fetch_monthly_close(
    ticker: str,
    start_date: Optional[Union[str, datetime]] = None,
    end_date:   Optional[Union[str, datetime]] = None
) -> pd.Series:
    """
    Fetch month-end closing prices for a given ticker from FMP.

    Uses the `/stable/historical-price-eod/full` endpoint with optional
    `from` and `to` parameters, then resamples to month-end.

    Args:
        ticker (str):       Stock or ETF symbol.
        start_date (str|datetime, optional): Earliest date (inclusive).
        end_date   (str|datetime, optional): Latest date (inclusive).

    Returns:
        pd.Series: Month-end close prices indexed by date.
    """
    params = {"symbol": ticker, "apikey": API_KEY, "serietype": "line"}
    if start_date:
        params["from"] = pd.to_datetime(start_date).date().isoformat()
    if end_date:
        params["to"]   = pd.to_datetime(end_date).date().isoformat()

    resp = requests.get(f"{BASE_URL}/historical-price-eod/full", params=params, timeout=30)
    resp.raise_for_status()
    raw  = resp.json()
    data = raw if isinstance(raw, list) else raw.get("historical", [])

    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["date"])
    df.set_index("date", inplace=True)
    monthly = df.sort_index().resample("ME")["close"].last()
    return monthly



In [ ]:
# File: run_portfolio_risk.py

import yaml
from pprint import pprint
from typing import Optional

def load_and_display_portfolio_config(filepath: str = "portfolio.yaml") -> Optional[dict]:
    """
    Loads the portfolio YAML file, parses weights, and prints input diagnostics.

    Args:
        filepath (str): Path to the YAML portfolio config file.

    Returns:
        dict: Parsed config dictionary (useful for testing or further calls).
    """
    with open(filepath, "r") as f:
        config = yaml.safe_load(f)

    # Extract components
    start_date            = config["start_date"]
    end_date              = config["end_date"]
    portfolio_input       = config["portfolio_input"]
    expected_returns      = config["expected_returns"]
    stock_factor_proxies  = config["stock_factor_proxies"]

    parsed  = standardize_portfolio_input(portfolio_input, latest_price)
    weights = parsed["weights"]

    # ─── Print outputs ─────────────────────────────────────
    print("=== Normalized Weights ===")
    print(weights)

    print("\n=== Dollar Exposure ===")
    print(parsed["dollar_exposure"])

    print("\n=== Total Portfolio Value ===")
    print(parsed["total_value"])

    print("\n=== Net Exposure (sum of weights) ===")
    print(parsed["net_exposure"])

    print("\n=== Gross Exposure (sum of abs(weights)) ===")
    print(parsed["gross_exposure"])

    print("\n=== Leverage (gross / net) ===")
    print(parsed["leverage"])

    print("\n=== Expected Returns ===")
    pprint(expected_returns)

    print("\n=== Stock Factor Proxies ===")
    for ticker, proxies in stock_factor_proxies.items():
        print(f"\n→ {ticker}")
        pprint(proxies)

    return config